<div align="center">
<p align="center" style="width: 100%;">
    <img src="https://raw.githubusercontent.com/vlm-run/.github/refs/heads/main/profile/assets/vlm-black.svg" alt="VLM Run Logo" width="80" style="margin-bottom: -5px; color: #2e3138; vertical-align: middle; padding-right: 5px;"><br>
</p>
<p align="center"><a href="https://docs.vlm.run"><b>Website</b></a> | <a href="https://docs.vlm.run/"><b>API Docs</b></a> | <a href="https://docs.vlm.run/blog"><b>Blog</b></a> | <a href="https://discord.gg/AMApC2UzVY"><b>Discord</b></a> | <a href="https://chat.vlm.run"><b>Chat</b></a>
</p>
</div>

# VLM Run Orion - UI Sketch to Functional React/Tailwind Code

This notebook demonstrates how to use [VLM Run Orion's](https://vlm.run/orion) vision capabilities to convert handwritten UI wireframes and sketches into clean, responsive, and functional React components using Tailwind CSS. This is particularly useful for quickly prototyping frontend interfaces from rough wireframes drawn on napkins, whiteboards, or paper.

For more details on the API, see the [Agent API docs](https://docs.vlm.run/agents/introduction).

## Prerequisites

- Python 3.10+
- VLM Run API key (get one at [app.vlm.run](https://app.vlm.run))
- VLM Run Python Client with OpenAI extra `vlmrun[openai]`


## Setup

First, install the required packages and configure the environment.


In [1]:
# Install required packages
%pip install vlmrun[openai] --upgrade --quiet
%pip install cachetools pillow requests numpy --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import getpass

VLMRUN_API_KEY = os.getenv("VLMRUN_API_KEY", None)
if VLMRUN_API_KEY is None:
    VLMRUN_API_KEY = getpass.getpass("Enter your VLM Run API key: ")


## Initialize the VLM Run Client

We use the OpenAI-compatible chat completions interface through the VLM Run SDK.


In [3]:
from vlmrun.client import VLMRun

BASE_URL = os.getenv("VLMRUN_BASE_URL", "https://agent.vlm.run/v1")
client = VLMRun(api_key=VLMRUN_API_KEY, base_url=BASE_URL)
print("VLM Run client initialized successfully!")
print(f"Base URL: {BASE_URL}")


VLM Run client initialized successfully!
Base URL: https://agent.vlm.run/v1


## Response Models

We define Pydantic models for structured outputs. The response will include the React component code for the UI sketch.


In [4]:
from pydantic import BaseModel, Field


class ReactCodeResponse(BaseModel):
    """Response containing the React component code for the UI sketch."""
    react_code: str = Field(..., description="The complete, functional React component code with Tailwind CSS styling")

print("Response models defined successfully!")


Response models defined successfully!


## Helper Functions

We create helper functions to simplify making chat completion requests with structured outputs.


In [5]:
import hashlib
import json
from typing import Any, Type, TypeVar

import cachetools
from vlmrun.common.image import encode_image
from PIL import Image


T = TypeVar('T', bound=BaseModel)


def custom_key(prompt: str, image_path: str | None = None, response_model: Type[T] | None = None, model: str = "vlmrun-orion-1:auto"):
    """Custom key for caching chat_completion."""
    response_key = hashlib.sha256(json.dumps(response_model.model_json_schema(), sort_keys=True).encode()).hexdigest() if response_model else ""
    image_key = hashlib.sha256(image_path.encode()).hexdigest() if image_path else ""
    return (prompt, image_key, response_key, model)


@cachetools.cached(cache=cachetools.TTLCache(maxsize=100, ttl=3600), key=custom_key)
def chat_completion(
    prompt: str,
    image_path: str | None = None,
    response_model: Type[T] | None = None,
    model: str = "vlmrun-orion-1:auto"
) -> tuple[BaseModel | str, str]:
    """
    Make a chat completion request with structured output for React code generation.

    Args:
        prompt: The prompt describing the React code generation task
        image_path: Path to the image file containing the UI sketch
        response_model: Pydantic model for structured output
        model: Model to use (default: vlmrun-orion-1:auto)

    Returns:
        Tuple of (parsed response model or text, session_id)
    """
    content = [{"type": "text", "text": prompt}]
    
    # Add image if provided
    if image_path:
        image = Image.open(image_path)
        image_data = encode_image(image, format="JPEG")
        content.append({"type": "image_url", "image_url": {"url": image_data}})

    kwargs = {
        "model": model,
        "messages": [{"role": "user", "content": content}]
    }

    if response_model:
        kwargs["response_format"] = {
            "type": "json_schema",
            "schema": response_model.model_json_schema()
        }

    response = client.agent.completions.create(**kwargs)
    response_text = response.choices[0].message.content
    session_id = response.session_id

    if response_model:
        result = response_model.model_validate_json(response_text)
        return result, session_id

    return response_text, session_id

print("Helper functions defined!")


Helper functions defined!


## UI Sketch to React/Tailwind Conversion

Convert a handwritten UI wireframe into clean, responsive, and functional React component code. Provide the path to an image file containing the UI sketch.


In [6]:
# Combined prompt for React code generation
REACT_CODE_PROMPT = """
Task: Act as an expert Frontend Engineer. Convert the attached handwritten UI wireframe into a clean, responsive, and functional React component using Tailwind CSS.

Instructions:

Component Architecture: Create a single-file functional React component. Use lucide-react for icons and framer-motion for basic entrance animations if applicable.

Styling (Tailwind CSS):
- Translate the rough layout into a polished, modern design.
- Ensure full responsiveness (mobile-first approach).
- Implement a professional color palette (e.g., slate/gray scales with a primary accent color) unless specific colors are noted in the sketch.

Interactivity:
- Use React useState for any interactive elements visible in the sketch (e.g., toggles, input fields, or dropdowns).
- Include hover states for buttons and links.

Code Quality:
- Use semantic HTML5 tags (<header>, <main>, <section>, <footer>).
- Ensure the code is modular, well-commented, and ready to be dropped into a Vite or Next.js project.
"""

print("React code generation prompt prepared!")
print(f"\nPrompt length: {len(REACT_CODE_PROMPT)} characters")


React code generation prompt prepared!

Prompt length: 991 characters


In [ ]:
# Example: Convert UI sketch to React code
# Replace 'path/to/your/ui_sketch.jpg' with the actual path to your image
image_path = "sketch.webp"

# Generate the React code
result, session_id = chat_completion(
    prompt=REACT_CODE_PROMPT,
    image_path=image_path,
    response_model=ReactCodeResponse,
    model="vlmrun-orion-1:auto"
)

print(">> RESPONSE")
print(result)
print(f"\n>> SESSION ID: {session_id}")
print("\n>> GENERATED REACT CODE")
print("=" * 80)
print(result.react_code)
print("=" * 80)


>> RESPONSE
react_code='import React, { useState } from \'react\';\nimport { \n  Menu, \n  Search, \n  Heart, \n  Star, \n  User, \n  Play, \n  Share2, \n  MoreVertical,\n  Bell,\n  X\n} from \'lucide-react\';\nimport { motion, AnimatePresence } from \'framer-motion\';\n\nconst VideoPlatform = () => {\n  const [searchQuery, setSearchQuery] = useState(\'\');\n  const [isLiked, setIsLiked] = useState(false);\n  const [isStarred, setIsStarred] = useState(false);\n  const [isPlaying, setIsPlaying] = useState(false);\n\n  // Animation variants\n  const fadeInUp = {\n    initial: { opacity: 0, y: 20 },\n    animate: { opacity: 1, y: 0 },\n    transition: { duration: 0.5 }\n  };\n\n  const staggerContainer = {\n    animate: {\n      transition: {\n        staggerChildren: 0.1\n      }\n    }\n  };\n\n  return (\n    <div className="min-h-screen bg-slate-50 text-slate-900 font-sans selection:bg-indigo-100 selection:text-indigo-900">\n      {/* Header Section */}\n      <header className="stick

---

## Conclusion

This notebook demonstrated how to use **VLM Run Orion** to convert handwritten UI wireframes into clean, responsive, and functional React components with Tailwind CSS.

### Key Takeaways

1. **Structured Prompts**: Well-structured prompts with clear instructions for component architecture, styling, interactivity, and code quality produce better results.
2. **Session Management**: Use `session_id` to track your code generation requests.
3. **Image Input**: Provide clear, high-quality images of UI sketches for best results.
4. **React Output**: The model returns production-ready React code that can be directly used in Vite or Next.js projects.

### Next Steps

- Experiment with different types of UI sketches (dashboards, forms, landing pages, etc.)
- Try different styling approaches and component architectures
- Explore the [VLM Run Documentation](https://docs.vlm.run) for more capabilities
- Join our [Discord community](https://discord.gg/AMApC2UzVY) for support

Happy coding! 🚀
